In [12]:
import random
import timeit
from copy import copy

TIME_LIMIT_MILLIS = 150

class Board(object):
    '''
    Implement a classic Tic Tac Toe board
    
    Paremeters
    ------------
    player_1: object
        object with get_move() function
    player_2: object
        object with get_move() function
        
    width : int (optional) number of columns in board
    height : int (optional) number of rows in board
    
    '''
    BLANK = 0
    NOT_MOVED = None
    def __init__(self, player1, player2, width=3, height=3):
        #board and player initialization
        self.width = width
        self.height = height
        self.move_count = 0
        self._player1 = player1
        self._player2 = player2
        self._active_player = player1
        self._inactive_player = player2
        
        #initiative and last move
        self._board_state = [Board.BLANK] * (width * height + 3)
        self._board_state[-1] = Board.NOT_MOVED
        self._board_state[-2] = Board.NOT_MOVED
        
    def active_player(self):
        '''
        Return player object holding initiative
        '''
        return self._active_player
        
    def inactive_player(self):
        '''
        return player object in waiting
        '''
        return self._inactive_player
    
    def get_opponent(self, player):
        '''
        Parameters
        ------------
        player: object register as a player in current game
        
        returns
        --------
        object representing opponent of input player
        '''
        if player == self._active_player:
            return self._inactive_player
        elif player == self._inactive_player:
            return self._active_player
        raise RuntimeError("player must be a registered object in current game")

    def copy(self):
        '''
        Return a deep copy of current board
        '''
        new_board = Board(self._player1, self._player2, width=self.width, height=self.height)
        new_board.move_count = self.move_count
        new_board._active_player = self._active_player
        new_board._inactive_player = self._inactive_player
        new_board._board_state = copy(self._board_state)
        return new_board
    
    def forecast_move(self, move):
        '''
        Return a deep copy of current game with move applied
        
        Parameters
        ------------
        move: (int,int)
            coordinate pair indicating next position for active player
        returns game board
        '''
        new_board = self.copy()
        new_board.apply_move(move)
        return new_board
    
    def is_legal_move(self, move):
        '''
        Test if input move is legal in current game state
        
        Parameters
        -----------
        move: (int,int) coordinate pair inidicating next position for active player
        
        returns
        ----------
        bool of if move is legal
        '''
        
        idx = move[0] + movie[1]  * self.height
        return (0<=move[0]< self.height and 0 <=move[1] < self.width and self._board_state[idx] == Board.BLANK)

    def get_blank_spaces(self):
        '''
        return all open spaces
        '''
        return [(i,j) for j in range(self.width) for i in range(self.height) if self._board_state[i+j*self.height] == Board.BLANK]
    def get_player_location(self,player):
        ''' find current location of input player
        Parameters
        ----------
        player: object representing player in current game
        
        Returns
        -------
        (int,int) or None
            coordinate pair of location of input player
        '''
        if player == self._player1:
            if self._board_state[-1] == Board.NOT_MOVED:
                return Board.NOT_MOVED
            idx = self._board_state[-1]
        if player == self._player2:
            if self._board_state[-2] == Board.NOT_MOVED:
                return Board.NOT_MOVED
            idx = self._board_state[-2]
        else:
            raise RuntimeError('Invalid player in get_player_location: {}'.format(player))
        w = idx // self.height
        h = idx % self.height
        return(h,w)
    
    def get_legal_moves(self, player=None):
        '''Returns list of all legal moves for specified player
        Parameters
        ----------
        player: object representing a registered player
        
        returns
        -------
        list of <(int),(int)> repreenting all legal moves for player
        '''
        if player is None:
            player = self.active_player
        return self.get_blank_spaces()

    def apply_move(self, move):
        '''Move player to specified location
        Parameters
        ----------
        move: (int,int)
            A coordinate pair indicating next position for active player
        '''
        
        idx = move[0] + move[1] * self.height
        last_move_idx = int(self._active_player == self._player2) + 1
        self._board_state[-last_move_idx] = idx
        self._board_state[idx] = last_move_idx
        #self._board_state[-3] ^=1
        self._active_player, self._inactive_player = self._inactive_player, self._active_player
        self.move_count += 1
        
    def is_winner(self, player):
        '''Test is input player won the game'''
        return player == self._inactive_player and not self.get_legal_moves(self._active_player)
    
    def is_loser(self,player):
        '''Test is input player lost the game'''
        return player == self._active_player and not self.get_legal_moves(self._active_player)
        
    def utility(self, player):
        """Returns the utility of the current game state from the perspective
        of the specified player.

                    /  +infinity,   "player" wins
        utility =  |   -infinity,   "player" loses
                    \          0,    otherwise

        Parameters
        ----------
        player : object (optional)
            An object registered as a player in the current game. If None,
            return the utility for the active player on the board.

        Returns
        ----------
        float
            The utility value of the current game state for the specified
            player. The game has a utility of +inf if the player has won,
            a value of -inf if the player has lost, and a value of 0
            otherwise.
        """
        if not self.get_legal_moves(self.active_player):
            if player == self._inactive_player:
                return float("inf")
            if player == self._active_player:
                return float("-inf")
            return 0.
        
    def to_string(self, symbols=['X', 'O']):
        """Generate a string representation of the current game state, marking
        the location of each player and indicating which cells have been
        blocked, and which remain open.
        """
        p1_loc = self._board_state[-1]
        p2_loc = self._board_state[-2]

        col_margin = len(str(self.height - 1)) + 1
        prefix = "{:<" + "{}".format(col_margin) + "}"
        offset = " " * (col_margin + 3)
        out = offset + '   '.join(map(str, range(self.width))) + '\n\r'
        for i in range(self.height):
            out += prefix.format(i) + ' | '
            for j in range(self.width):
                idx = i + j * self.height
                if not self._board_state[idx]:
                    out += ' '
                else:
                    out += symbols[int(self._board_state[idx]-1)]
                out += ' | '
            out += '\n\r'

        return out
    
    def play(self, time_limit=TIME_LIMIT_MILLIS):
        """Execute a match between the players by alternately soliciting them
        to select a move and applying it in the game.

        Parameters
        ----------
        time_limit : numeric (optional)
            The maximum number of milliseconds to allow before timeout
            during each turn.

        Returns
        ----------
        (player, list<[(int, int),]>, str)
            Return multiple including the winning player, the complete game
            move history, and a string indicating the reason for losing
            (e.g., timeout or invalid move).
        """
        move_history = []

        time_millis = lambda: 1000 * timeit.default_timer()

        while True:

            legal_player_moves = self.get_legal_moves()
            game_copy = self.copy()

            move_start = time_millis()
            time_left = lambda : time_limit - (time_millis() - move_start)
            curr_move = self._active_player.get_move(game_copy, time_left)
            move_end = time_left()

            if curr_move is None:
                curr_move = Board.NOT_MOVED

            if move_end < 0:
                return self._inactive_player, move_history, "timeout"

            if curr_move not in legal_player_moves:
                if len(legal_player_moves) > 0:
                    return self._inactive_player, move_history, "forfeit"
                return self._inactive_player, move_history, "illegal move"

            move_history.append(list(curr_move))

            self.apply_move(curr_move)    
    

In [25]:
# game = Board("player1","player2")

# game.apply_move((1,1))
# game.apply_move((0,0))
# game.apply_move((0,2))
# game.apply_move((1,0))
# game.apply_move((2,0))

# print(game.to_string())

def cross(A, B):
    "Cross product of elements in A and elements in B."
    return [(s,t) for s in A for t in B]

rows = range(3)
cols = range(3)
row_units = [cross(r,cols) for r in rows]
column_units = [cross(rows, c) for c in cols]

print(row_units)
print(column_units)

TypeError: 'int' object is not iterable